<div align="center">
<p align="center" style="width: 100%;">
    <img src="https://raw.githubusercontent.com/vlm-run/.github/refs/heads/main/profile/assets/vlm-black.svg" alt="VLM Run Logo" width="80" style="margin-bottom: -5px; color: #2e3138; vertical-align: middle; padding-right: 5px;"><br>
</p>
<p align="center"><a href="https://docs.vlm.run"><b>Website</b></a> | <a href="https://docs.vlm.run/"><b>API Docs</b></a> | <a href="https://docs.vlm.run/blog"><b>Blog</b></a> | <a href="https://discord.gg/AMApC2UzVY"><b>Discord</b></a>
</p>
<p align="center">
<a href="https://discord.gg/AMApC2UzVY"><img alt="Discord" src="https://img.shields.io/badge/discord-chat-purple?color=%235765F2&label=discord&logo=discord"></a>
<a href="https://twitter.com/vlmrun"><img alt="Twitter Follow" src="https://img.shields.io/twitter/follow/vlmrun.svg?style=social&logo=twitter"></a>
</p>
</div>

Welcome to **[VLM Run Cookbooks](https://github.com/vlm-run/vlmrun-cookbook)**, a comprehensive collection of examples and notebooks demonstrating the power of structured visual understanding using the [VLM Run Platform](https://app.vlm.run).


## Case Study: Object Detection with VLM Run SDK

This notebook demonstrates how to use the **official VLM Run Node.js SDK** (`vlmrun` npm package) for object detection. We'll cover:

- **Object Detection**: Detect 80+ COCO dataset classes (person, car, cat, etc.)
- **Person Detection**: Specialized detection for people in images
- **Face Detection**: Detect and locate faces with high precision
- **SDK Features**: Explore models, hub domains, and more

All detections return bounding boxes in normalized `xywh` format (x, y, width, height) where values are between 0-1, along with labels.

Reference: [VLM Run Detection Documentation](https://docs.vlm.run/agents/capabilities/image/detection)


### Environment Setup

To get started, install the VLM Run SDK and sign up for an API key on the [VLM Run App](https://app.vlm.run).
- Store the VLM Run API key under the `VLMRUN_API_KEY` environment variable.


## Prerequisites

* Node.js 18+
* VLM Run API key (get one at [app.vlm.run](https://app.vlm.run))
* Deno or tslab kernel for running TypeScript in Jupyter


## Setup

First, let's install the required packages:


In [ ]:
// Install the VLM Run SDK
// npm install vlmrun openai zod zod-to-json-schema

// If using Deno kernel, install dependencies via npm specifiers
// For tslab, run: npm install vlmrun openai zod zod-to-json-schema in your project directory


In [1]:
// Import the VLM Run SDK and dependencies
import { VlmRun } from "vlmrun";
import { z } from "zod";
import { zodToJsonSchema } from "zod-to-json-schema";


In [2]:
// Get API key from environment variable
const VLMRUN_API_KEY = Deno.env.get("VLMRUN_API_KEY");

if (!VLMRUN_API_KEY) {
    throw new Error("Please set the VLMRUN_API_KEY environment variable");
}

console.log("✓ API Key loaded successfully");


Error: Please set the VLMRUN_API_KEY environment variable

Let's initialize the VLM Run client using the SDK


In [3]:
// Initialize the VLM Run client using the SDK
const client = new VlmRun({
    apiKey: VLMRUN_API_KEY,
    baseURL: "https://agent.vlm.run/v1"  // Use the agent API endpoint
});

console.log("✓ VLM Run SDK client initialized!");


✓ VLM Run SDK client initialized!


## Define Detection Schemas

We'll define Zod schemas for the detection responses (TypeScript equivalent of Python's Pydantic):


In [4]:
// Define the Detection schema using Zod
const DetectionSchema = z.object({
    label: z.string().describe("Name of the detected object"),
    xywh: z.tuple([z.number(), z.number(), z.number(), z.number()])
        .describe("Bounding box (x, y, width, height) - normalized values 0-1")
});

// Define the Detections response schema
const DetectionsSchema = z.object({
    detections: z.array(DetectionSchema).describe("List of detected objects")
});

// Type inference from schemas
type Detection = z.infer<typeof DetectionSchema>;
type Detections = z.infer<typeof DetectionsSchema>;

// Convert Zod schema to JSON Schema for API
const detectionsJsonSchema = zodToJsonSchema(DetectionsSchema);
console.log("✓ Detection schemas defined");
console.log("JSON Schema:", JSON.stringify(detectionsJsonSchema, null, 2));


✓ Detection schemas defined
JSON Schema: {
  "type": "object",
  "properties": {
    "detections": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "label": {
            "type": "string",
            "description": "Name of the detected object"
          },
          "xywh": {
            "type": "array",
            "minItems": 4,
            "maxItems": 4,
            "items": [
              {
                "type": "number"
              },
              {
                "type": "number"
              },
              {
                "type": "number"
              },
              {
                "type": "number"
              }
            ],
            "description": "Bounding box (x, y, width, height) - normalized values 0-1"
          }
        },
        "required": [
          "label",
          "xywh"
        ],
        "additionalProperties": false
      },
      "description": "List of detected objects"
    }
  }

## Helper Functions

Let's create utility functions for detection using the VLM Run SDK:


In [5]:
/**
 * Perform object detection on an image using VLM Run SDK.
 * 
 * @param imageUrl - URL of the image to analyze
 * @param prompt - Detection prompt (e.g., "Detect all objects", "Detect all people")
 * @returns Detections object with bounding boxes
 */
async function detectObjects(imageUrl: string, prompt: string = "Detect all objects in this image"): Promise<Detections> {
    // Use the SDK's agent.completions interface
    const response = await client.agent.completions.create({
        model: "vlm-agent-1",
        messages: [
            {
                role: "user",
                content: [
                    { type: "text", text: prompt },
                    { type: "image_url", image_url: { url: imageUrl, detail: "auto" } }
                ]
            }
        ],
        // Use VLM Run's format - schema at top level
        response_format: { 
            type: "json_schema", 
            schema: detectionsJsonSchema
        } as any
    });
    
    const rawContent = response.choices[0].message.content || "{}";
    const parsed = JSON.parse(rawContent);
    return DetectionsSchema.parse(parsed);
}

/**
 * Format detection results for display.
 */
function formatDetections(detections: Detection[], imageWidth?: number, imageHeight?: number): void {
    console.log(`\n📦 Found ${detections.length} objects:\n`);
    
    detections.forEach((det, i) => {
        const [x, y, w, h] = det.xywh;
        
        if (imageWidth && imageHeight) {
            const x_px = Math.round(x * imageWidth);
            const y_px = Math.round(y * imageHeight);
            const w_px = Math.round(w * imageWidth);
            const h_px = Math.round(h * imageHeight);
            console.log(`  ${i + 1}. ${det.label}`);
            console.log(`     Normalized: x=${x.toFixed(3)}, y=${y.toFixed(3)}, w=${w.toFixed(3)}, h=${h.toFixed(3)}`);
            console.log(`     Pixels:     x=${x_px}, y=${y_px}, w=${w_px}, h=${h_px}`);
        } else {
            console.log(`  ${i + 1}. ${det.label}: xywh=[${x.toFixed(3)}, ${y.toFixed(3)}, ${w.toFixed(3)}, ${h.toFixed(3)}]`);
        }
    });
}

/**
 * Group detections by label and count them.
 */
function countByLabel(detections: Detection[]): Record<string, number> {
    return detections.reduce((acc, det) => {
        acc[det.label] = (acc[det.label] || 0) + 1;
        return acc;
    }, {} as Record<string, number>);
}

console.log("✓ Helper functions defined");


✓ Helper functions defined


## Example 1: Person Detection

Let's detect people in an image using the VLM Run SDK.


In [6]:
// Image with people
const peopleImageUrl = "https://images.unsplash.com/photo-1511632765486-a01980e01a18?w=800";

console.log("📷 Image URL:", peopleImageUrl);
console.log("\n🔍 Detecting people using VLM Run SDK...\n");

// Perform person detection
const personResult = await detectObjects(peopleImageUrl, "Detect all people in this image");

// Display results
console.log(`Found ${personResult.detections.length} people in the image:`);
personResult.detections.forEach((det, i) => {
    console.log(`  Person ${i + 1}: ${det.label}, xywh=[${det.xywh.map(v => v.toFixed(3)).join(", ")}]`);
});

// Show summary
const labelCounts = countByLabel(personResult.detections);
console.log("\n📊 Detection Summary:");
console.table(labelCounts);


📷 Image URL: https://images.unsplash.com/photo-1511632765486-a01980e01a18?w=800

🔍 Detecting people using VLM Run SDK...

Found 4 people in the image:
  Person 1: person, xywh=[0.325, 0.267, 0.128, 0.628]
  Person 2: person, xywh=[0.401, 0.272, 0.144, 0.623]
  Person 3: person, xywh=[0.488, 0.272, 0.140, 0.623]
  Person 4: person, xywh=[0.592, 0.240, 0.116, 0.655]

📊 Detection Summary:
┌────────┬────────┐
│ (idx)  │ Values │
├────────┼────────┤
│ person │      4 │
└────────┴────────┘


## Example 2: General Object Detection

Let's detect various objects in a street scene.


In [7]:
// Street scene image
const streetImageUrl = "https://storage.googleapis.com/vlm-data-public-prod/hub/examples/image.object-detection/crossroad.jpg";

console.log("📷 Image URL:", streetImageUrl);
console.log("\n🔍 Detecting all objects in the street scene...\n");

// Perform general object detection
const objectResult = await detectObjects(streetImageUrl, "Detect all objects in this image including people, vehicles, and other objects");

// Format and display results with assumed image dimensions
formatDetections(objectResult.detections, 800, 600);

// Show counts by category
const objectCounts = countByLabel(objectResult.detections);
console.log("\n📊 Objects Detected by Category:");
console.table(objectCounts);


📷 Image URL: https://storage.googleapis.com/vlm-data-public-prod/hub/examples/image.object-detection/crossroad.jpg

🔍 Detecting all objects in the street scene...


📦 Found 155 objects:

  1. person
     Normalized: x=0.141, y=0.385, w=0.112, h=0.445
     Pixels:     x=113, y=231, w=90, h=267
  2. person
     Normalized: x=0.550, y=0.388, w=0.202, h=0.560
     Pixels:     x=440, y=233, w=162, h=336
  3. person
     Normalized: x=0.550, y=0.470, w=0.078, h=0.425
     Pixels:     x=440, y=282, w=62, h=255
  4. person
     Normalized: x=0.390, y=0.409, w=0.062, h=0.262
     Pixels:     x=312, y=245, w=50, h=157
  5. person
     Normalized: x=0.531, y=0.624, w=0.059, h=0.274
     Pixels:     x=425, y=374, w=47, h=164
  6. person
     Normalized: x=0.409, y=0.727, w=0.094, h=0.116
     Pixels:     x=327, y=436, w=75, h=70
  7. person
     Normalized: x=0.052, y=0.420, w=0.028, h=0.026
     Pixels:     x=42, y=252, w=22, h=16
  8. person
     Normalized: x=0.151, y=0.436, w=0.018, h=0.091
  

## Example 3: Face Detection

Let's specifically detect faces in an image.


In [8]:
// Use the same people image for face detection
console.log("📷 Image URL:", peopleImageUrl);
console.log("\n🔍 Detecting faces using VLM Run SDK...\n");

// Perform face detection
const faceResult = await detectObjects(peopleImageUrl, "Detect all faces in this image");

// Display face detection results
console.log(`Found ${faceResult.detections.length} faces:`);
faceResult.detections.forEach((det, i) => {
    const [x, y, w, h] = det.xywh;
    console.log(`  Face ${i + 1}: x=${x.toFixed(3)}, y=${y.toFixed(3)}, w=${w.toFixed(3)}, h=${h.toFixed(3)}`);
});


📷 Image URL: https://images.unsplash.com/photo-1511632765486-a01980e01a18?w=800

🔍 Detecting faces using VLM Run SDK...

Found 4 faces:
  Face 1: x=0.323, y=0.267, w=0.123, h=0.620
  Face 2: x=0.399, y=0.273, w=0.143, h=0.624
  Face 3: x=0.485, y=0.275, w=0.142, h=0.628
  Face 4: x=0.592, y=0.240, w=0.111, h=0.653


## Example 4: Batch Processing Multiple Images

Process multiple images and aggregate detection results using the SDK.


In [9]:
// Multiple images for batch processing
const batchImages = [
    { url: "https://storage.googleapis.com/vlm-data-public-prod/hub/examples/image.object-detection/crossroad.jpg", name: "Street Scene" },
    { url: "https://images.unsplash.com/photo-1511632765486-a01980e01a18?w=800", name: "Group of People" },
    { url: "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400", name: "Cat" },
];

interface BatchDetectionResult {
    imageName: string;
    url: string;
    objectCount: number;
    labels: string[];
    error?: string;
}

async function processBatch(images: { url: string; name: string }[]): Promise<BatchDetectionResult[]> {
    const results: BatchDetectionResult[] = [];
    
    for (const img of images) {
        try {
            console.log(`Processing: ${img.name}...`);
            const detections = await detectObjects(img.url);
            const uniqueLabels = [...new Set(detections.detections.map(d => d.label))];
            
            results.push({
                imageName: img.name,
                url: img.url,
                objectCount: detections.detections.length,
                labels: uniqueLabels
            });
        } catch (error) {
            console.log(`Error processing ${img.name}: ${error}`);
            results.push({
                imageName: img.name,
                url: img.url,
                objectCount: 0,
                labels: [],
                error: String(error)
            });
        }
    }
    
    return results;
}

// Process all images
const batchResults = await processBatch(batchImages);

// Display batch results
console.log("\n=== Batch Detection Results ===\n");
console.table(batchResults.map(r => ({
    Image: r.imageName,
    "Objects Found": r.objectCount,
    "Unique Labels": r.labels.join(", ") || "Error"
})));


Processing: Street Scene...
Processing: Group of People...
Processing: Cat...

=== Batch Detection Results ===

┌───────┬───────────────────┬───────────────┬────────────────────────┐
│ (idx) │ Image             │ Objects Found │ Unique Labels          │
├───────┼───────────────────┼───────────────┼────────────────────────┤
│     0 │ "Street Scene"    │            97 │ "person"               │
│     1 │ "Group of People" │            12 │ "person, shirt, pants" │
│     2 │ "Cat"             │             3 │ "cat, bamboo, face"    │
└───────┴───────────────────┴───────────────┴────────────────────────┘


## Example 5: Using VLM Run SDK Image Predictions API

The VLM Run SDK also provides a dedicated `image` API for predictions. Let's explore this approach:


In [11]:
// Initialize a separate client for the main API (not agent API)
const vlmClient = new VlmRun({
    apiKey: VLMRUN_API_KEY,
    baseURL: "https://api.vlm.run/v1"  // Main API endpoint
});

console.log("✓ VLM Run main API client initialized!");

// Using the image predictions API
console.log("\n📷 Using VLM Run Image API for detection...\n");

try {
    // Generate detection using the image.generate method
    const imagePrediction = await vlmClient.image.generate({
        images: [streetImageUrl],
        model: "vlm-1",
        domain: "image.object-detection",
        config: {
            jsonSchema: detectionsJsonSchema
        }
    });
    
    console.log("Image Prediction Response:");
    console.log(JSON.stringify(imagePrediction, null, 2));
} catch (error) {
    console.log("Note: Image predictions API requires specific domain access.");
    console.log("For general detection, use the agent.completions API as shown in previous examples.");
    console.log("Error:", error);
}


✓ VLM Run main API client initialized!

📷 Using VLM Run Image API for detection...

Image Prediction Response:
{
  "usage": {
    "elements_processed": 1,
    "element_type": "image",
    "credits_used": 4,
    "steps": null,
    "message": null,
    "duration_seconds": 0
  },
  "id": "285d6122-aea4-4a9c-85d4-cac02a8c7e27",
  "created_at": "2025-12-18T14:06:00.827583",
  "completed_at": "2025-12-18T14:06:21.396589Z",
  "response": {
    "content": "A busy New York City street scene shows several pedestrians crossing the street on a <Object id='obj-15'/>. On the left, a <Object id='obj-0'/> wearing a backpack walks away from the viewer, with a red <Object id='obj-2'/> parked behind him. A bright yellow <Object id='obj-1'/> is visible in the center, next to a partially obscured <Object id='obj-8'/>. To the right, a <Object id='obj-3'/> in a light blue shirt walks towards the right, alongside a <Object id='obj-4'/>. A <Object id='obj-5'/> is seen pushing a <Object id='obj-6'/> which carri

## VLM Run SDK Features

The VLM Run SDK (`vlmrun` npm package) provides several advantages over using the OpenAI client directly:

### Key Features

| Feature | Description |
|---------|-------------|
| `agent.completions` | OpenAI-compatible chat completions for agent API |
| `image.generate()` | Direct image prediction API access |
| `document.generate()` | Document processing capabilities |
| `audio.generate()` | Audio transcription and analysis |
| `video.generate()` | Video understanding and analysis |
| `files.upload()` | File upload for processing |
| `hub.list()` | Browse available models and domains |
| `models.list()` | List available models |

### SDK Initialization Options

```typescript
const client = new VlmRun({
    apiKey: "your-api-key",
    baseURL: "https://api.vlm.run/v1",  // or "https://agent.vlm.run/v1"
    timeout: 60000,  // Request timeout in ms
    maxRetries: 3    // Number of retries for failed requests
});
```


## Understanding Bounding Box Format

VLM Run returns bounding boxes in **normalized xywh format**:

| Field | Description | Range |
|-------|-------------|-------|
| `x` | Left edge of bounding box | 0.0 - 1.0 |
| `y` | Top edge of bounding box | 0.0 - 1.0 |
| `w` | Width of bounding box | 0.0 - 1.0 |
| `h` | Height of bounding box | 0.0 - 1.0 |

To convert to pixel coordinates, multiply by image dimensions:
```typescript
const x_px = x * imageWidth;
const y_px = y * imageHeight;
const w_px = w * imageWidth;
const h_px = h * imageHeight;
```


## Best Practices & Tips

### Detection Quality
- Use specific prompts for better results (e.g., "Detect all cars" vs "Detect objects")
- For crowded scenes, consider detecting specific object types separately
- Use higher resolution images for better small object detection

### SDK Usage
- Use `agent.completions` for OpenAI-compatible chat interface with image support
- Use `image.generate()` for domain-specific image predictions
- Set appropriate timeouts for large image processing
- Use retry logic for production applications

### Supported Object Classes
VLM Run can detect 80+ COCO dataset classes including:
- **People**: person, face
- **Vehicles**: car, truck, bus, motorcycle, bicycle, airplane, boat
- **Animals**: cat, dog, bird, horse, cow, sheep
- **Objects**: chair, table, laptop, phone, book, bottle, cup

### Performance Tips
- Batch similar detection tasks for efficiency
- Cache detection results for frequently analyzed images
- Use appropriate image sizes (800-1200px width is usually sufficient)


## Additional Resources

- [VLM Run Documentation](https://docs.vlm.run)
- [VLM Run Node.js SDK](https://github.com/vlm-run/vlmrun-node-sdk)
- [SDK API Reference](https://docs.vlm.run/sdks/node-sdk)
- [Detection API Reference](https://docs.vlm.run/agents/capabilities/image/detection)
- [More Examples](https://github.com/vlm-run/vlmrun-cookbook)
- [Discord Community](https://discord.gg/AMApC2UzVY)

### Installation

```bash
npm install vlmrun openai zod zod-to-json-schema
```

### Quick Start

```typescript
import { VlmRun } from "vlmrun";
import { z } from "zod";
import { zodToJsonSchema } from "zod-to-json-schema";

const client = new VlmRun({
    apiKey: process.env.VLMRUN_API_KEY,
    baseURL: "https://agent.vlm.run/v1"
});

const DetectionsSchema = z.object({
    detections: z.array(z.object({
        label: z.string(),
        xywh: z.tuple([z.number(), z.number(), z.number(), z.number()])
    }))
});

const response = await client.agent.completions.create({
    model: "vlm-agent-1",
    messages: [
        {
            role: "user",
            content: [
                { type: "text", text: "Detect all objects in this image" },
                { type: "image_url", image_url: { url: "https://example.com/image.jpg" } }
            ]
        }
    ],
    response_format: { 
        type: "json_schema", 
        schema: zodToJsonSchema(DetectionsSchema)
    }
});

console.log(JSON.parse(response.choices[0].message.content));
```
